In [1]:
from keras import models, layers
from keras.datasets import mnist
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# load the data and devided into 2 sets call train and test data sets 
# training set consists of 60,000 28x28 pixel images, and the test set 10,000.
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
train_images.shape, test_images.shape

((60000, 28, 28), (10000, 28, 28))

In [4]:
# initializing the sequential model
model = models.Sequential()
# input layer
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
# hidden layers 
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
# output layer contains 10 neurons for each 0 - 9 digits 
model.add(layers.Dense(10, activation='softmax'))

# compiling the model 
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
# view model summary which we have created 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

In [6]:
# adding noise
noise_factor = 0.60
x_train_noisy = train_images + noise_factor * np.random.normal(loc=0.0,scale=1.0,size=train_images.shape)
x_test_noisy = test_images + noise_factor * np.random.normal(loc=0.0,scale=1.0,size=test_images.shape)
x_train_noisy = np.clip(x_train_noisy,0.,1.)
x_test_noisy = np.clip(x_test_noisy,0.,1.)

In [7]:
train_images = x_train_noisy
test_images = x_test_noisy

In [8]:
# reshaping the x_train_noisy images 
x_train_noisy = x_train_noisy.reshape((60000, 28, 28, 1))
x_train_noisy= x_train_noisy.astype('float32') / 255 # rescale pixel values from range [0, 255] to [0, 1]

# reshaping the test images  
x_test_noisy = x_test_noisy.reshape((10000, 28, 28, 1))
x_test_noisy= x_test_noisy.astype('float32') / 255

# label encoding 
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# data partition for model validation
validation_images = x_train_noisy[50000:]
validation_labels = train_labels[50000:]

# data partition for model training 
x_train_noisy = x_train_noisy[:50000]
train_labels = train_labels[:50000]

# train the model
history = model.fit(x_train_noisy, train_labels, epochs=20, batch_size=64, validation_data=(validation_images, validation_labels))

Epoch 1/20
782/782 [==============================] - 3s 4ms/step - loss: 1.3950 - accuracy: 0.5138 - val_loss: 0.5400 - val_accuracy: 0.8330
Epoch 2/20
782/782 [==============================] - 3s 4ms/step - loss: 0.4540 - accuracy: 0.8579 - val_loss: 0.3668 - val_accuracy: 0.8842
Epoch 3/20
782/782 [==============================] - 3s 4ms/step - loss: 0.2757 - accuracy: 0.9142 - val_loss: 0.2042 - val_accuracy: 0.9398
Epoch 4/20
782/782 [==============================] - 3s 4ms/step - loss: 0.1856 - accuracy: 0.9412 - val_loss: 0.1533 - val_accuracy: 0.9535
Epoch 5/20
782/782 [==============================] - 3s 4ms/step - loss: 0.1418 - accuracy: 0.9557 - val_loss: 0.1270 - val_accuracy: 0.9611
Epoch 6/20
782/782 [==============================] - 3s 4ms/step - loss: 0.1179 - accuracy: 0.9631 - val_loss: 0.1024 - val_accuracy: 0.9687
Epoch 7/20
782/782 [==============================] - 3s 4ms/step - loss: 0.1008 - accuracy: 0.9682 - val_loss: 0.1167 - val_accuracy: 0.9664
Epoch 

In [9]:
# validating the model
test_loss, test_acc = model.evaluate(x_test_noisy, test_labels)
print('Accuracy:', test_acc)
print('Loss: ', test_loss)

313/313 [==============================] - 1s 2ms/step - loss: 0.0605 - accuracy: 0.9818
Accuracy: 0.9818000197410583
Loss:  0.06054188683629036
